In [2]:
!pip install transformers

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import files
uploaded = files.upload()
import io


Saving DF1.csv to DF1.csv


In [5]:
# data_7a = pd.read_csv("DF1_team7a.csv")
# data_7b = pd.read_csv("DF1_team7b.csv")
# data_7c = pd.read_csv("DF1_team7c.csv")
# data_7c = data_7c.dropna()
# DF1 = pd.concat([data_7a, data_7b, data_7c], ignore_index=True)
# DF1 = DF1.drop(columns=['Sub-category', 'URL'])
# DF1.to_csv('DF1.csv', index=False)
# files.download('DF1.csv')
DF1 = pd.read_csv("DF1.csv")
DF1.head(1818)

,Title,Category,Post Text
0,Testing requirements in toys and games category,Amazon Custom,My product is domino sets for adults. I am the...
1,Food and Drug Administration,Amazon Custom,"Hello, I am on my way to becoming an Amazon de..."
2,Images Storage for mass upload,Amazon Custom,When uploading products with the spreadsheet w...
3,Prepaid Return Label program email clarification,Amazon Custom,You may have received an email from Amazon inf...
4,Pulling Amazon Custom Personalizatoin Data fro...,Amazon Custom,The problem: Currently Amazon gives you links ...
...,...,...,...
1810,A suspended account which 20% from his profits...,Account Health,20% of the profits are for charity. Can I expe...
1811,Need help with account suspension Appeal,Account Health,"Hello members,\nI am requesting for some advic..."
1812,Restricted Product Policy Violations,Account Health,So I am a new seller and just noticed that i h...
1813,8k on hold - sourcing from Home Depot - inauth...,Account Health,"Hi,\nIm a new seller on Amazon and have been s..."


In [6]:
# DF1["Title"].value_counts()

In [7]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [8]:
tokenized = DF1["Post Text"].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=150)))

In [9]:
# max_len = 0
# for i in tokenized.values:
#     if len(i) > max_len:
#         max_len = len(i)

max_len = 150
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(1815, 150)

In [10]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1815, 150)

In [11]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [12]:
features = last_hidden_states[0][:,0,:].numpy()
labels = DF1["Category"]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [17]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
lr_clf.score(test_features, test_labels)

0.8502202643171806